In [31]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from IPython import display
from pathlib import *
import pandas as pd
import xarray as xr
from bokeh.palettes import all_palettes
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from climatools.plot.plot import *
from climatools.atm import *
output_notebook()

Loading BokehJS ...

# Band09 $H_2O$ mls cosz=1

```
!## band 9 (8200 - 14290)                       
real, parameter    :: vstar = 8200.0                     
integer, parameter :: nband = 609  
integer, parameter :: nv = 10000   
real, parameter    :: dv = 0.001 

integer   flgh2o, flgco2, flgo3, flgo2   
data      flgh2o, flgco2, flgo3, flgo2  
        /   1,      0,      0,      0 /  
        
parameter (cosz=1,rsfc=0.0)  
integer, parameter :: nref = 2  
integer, parameter :: max_ng = 15  
integer, parameter :: ng = 10  
integer, parameter :: nlayer = 75  
include '/chia_cluster/home/jackyu/radiation/crdnew-sw/atmosphere_profiles/mls75.pro'  
real, dimension(nref), parameter :: p_refs =(/ 300., 300. /)  
real, dimension(nref), parameter :: t_refs =(/ 250., 250. /) 
integer, dimension(nref), parameter :: ng_refs = (/4, 6/)  
integer, dimension(nref), parameter :: ng_adju = (/ -3, 0 /)  
data wgt / 0.95, 0.90, 5*0.50, 0.70, 0.85, 0.95 /  
integer, parameter :: option_klin = 1  
integer, parameter :: option_k_lookup = 0 ! 1-interpolation with ktable   
integer, parameter :: nl = 0   
integer, parameter :: nt = 0


```


In [32]:
PATH = Path('../crdnew-sw/band09_-_h2o_atmpro_-_mls_-_cosz_1/')

In [33]:
np.random.seed(30)

## CRD heating rate for each $g$

In [60]:
heat = pd.read_csv(PATH/'fort.401', header=None, sep=r'\s+')
heat.set_index(0, inplace=True)
heat.index.names = ['pressure']
heat.columns.names = ['g']

pltdata = [{'srs':xr.DataArray(heat[g]),
            'label':f'{g}',
            'color':np.random.choice(all_palettes['Category20'][15]),
            'alpha': np.random.uniform(.8, 1.),
            'line_width':np.random.uniform(2, 7),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for g in heat]

fig_linp = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heat', prange=(50, 1050))
fig_logp = plt_vert_profile_bokeh(pltdata=pltdata, y_axis_type='log', xlabel='heat', prange=(1e-2, 200))
show(gridplot(fig_linp, fig_logp, ncols=2))

## WGT heating rate for each $g$

In [62]:
heatg = pd.read_csv(PATH/'fort.400', header=None, sep=r'\s+')
heatg.set_index(0, inplace=True)
heatg.index.names = ['pressure']
heatg.columns.names = ['g']

pltdata = [{'srs':xr.DataArray(heatg[g]),
            'label':f'{g}',
            'color':np.random.choice(all_palettes['Category20'][15]),
            'alpha':np.random.uniform(.8, 1.),
            'line_width':np.random.uniform(2, 7),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for g in heatg]

fig_linp = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heatg', prange=(50, 1050))
fig_logp = plt_vert_profile_bokeh(pltdata=pltdata, y_axis_type='log', xlabel='heatg', prange=(1e-2, 200))
show(gridplot(fig_linp, fig_logp, ncols=2))

## (CRD vs WGT)  heating rate for each $g$

In [56]:
heatg = pd.read_csv(PATH/'fort.400', header=None, sep=r'\s+')
heatg.set_index(0, inplace=True)
heatg.index.names = ['pressure']
heatg.columns.names = ['g']

heat = pd.read_csv(PATH/'fort.401', header=None, sep=r'\s+')
heat.set_index(0, inplace=True)
heat.index.names = ['pressure']
heat.columns.names = ['g']

for g in heat.columns:
    pltdata = [{'srs':xr.DataArray(heatg[g]),
                'label':f'WGT g={g}',
                'color':all_palettes['Set1'][9][0],
                'alpha':.5,
                'line_width':6,
                'line_dash':'dotdash'},
               {'srs':xr.DataArray(heat[g]),
                'label':f'CRD g={g}',
                'color':all_palettes['Set1'][9][1],
                'alpha':1,
                'line_width':3,
                'line_dash':'solid'}]

    fig_liny = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='Heating Rate', y_axis_type='linear', prange=(50, 1050))
    fig_logy = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='Heating Rate', y_axis_type='log', prange=(1e-2, 200))
    show(gridplot(fig_liny, fig_logy, ncols=2))

## Bands-total heating 

In [69]:
heat = pd.read_csv(PATH/'fort.10', sep=r'\s+', header=None)
heat.set_index(0, inplace=True)
heat.index.names = ['pressure']
heat.columns = ['CRD', 'WGT']

pltdata = [{'srs':xr.DataArray(heat[m]),
            'label':f'{m}',
            'color':np.random.choice(all_palettes['Category10'][3]),
            'alpha':np.random.uniform(.5, .8),
            'line_width':np.random.uniform(3, 6),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for m in heat]

fig_lin = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heat', prange=(50, 1050))
fig_log = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heat', y_axis_type='log', prange=(1e-2, 200))
show(gridplot(fig_lin, fig_log, ncols=2))

## Flux for each g

In [48]:
def load_flux(fpath=None, name=None):
    '''
    Return flux for each g-group, either calculated by line-by-line (fort.403), 
    or by clirad-sw (fort.402), in a xarray.DataArray.
    '''
    df = pd.read_csv(fpath, sep=r'\s+', header=None)
    df.set_index(0, inplace=True)
    df.index.names = ['pressure']
    df.columns.names = ['g']
    da = xr.DataArray(df)
    da.name = name
    return da

In [54]:
dwgt = load_flux(fpath=PATH/'fort.402', name='wgt')
dcrd = load_flux(fpath=PATH/'fort.403', name='crd')
dwgt = dwgt.isel(pressure=[0, -1]).to_pandas()
dcrd = dcrd.isel(pressure=[0, -1]).to_pandas()
pd.concat([dwgt, dcrd, dwgt - dcrd], axis=0,
          keys=['WGT', 'CRD', 'WGT - CRD'])

g                          1         2         3          4          5   \
          pressure                                                        
WGT       0.0003     0.367355  1.729681  7.086743  20.315927  15.890630   
          1001.1125  0.000000  0.000000  0.000006   1.021347   5.182406   
CRD       0.0003     0.367355  1.729681  7.086743  20.315927  15.890630   
          1001.1125  0.000000  0.000000  0.018040   2.036553   5.154647   
WGT - CRD 0.0003     0.000000  0.000000  0.000000   0.000000   0.000000   
          1001.1125  0.000000  0.000000 -0.018034  -1.015207   0.027759   

g                           6          7          8          9           10  
          pressure                                                           
WGT       0.0003     20.966282  25.023096  29.811970  32.446861  288.781206  
          1001.1125  11.717588  18.656202  25.473664  29.892327  286.019366  
CRD       0.0003     20.966282  25.023096  29.811970  32.446861  288.781206  
          1001.1125  11.264541  18.113505  25.236823  29.825633  285.907942  
WGT - CRD 0.0003      0.000000   0.000000   0.000000   0.000000    0.000000  
          1001.1125   0.453047   0.542697   0.236841   0.066695    0.111424

## Bands-total flux

In [79]:
flux = pd.read_csv(PATH/'fort.9', sep=r'\s+', skiprows=2, header=None)
flux.set_index(0, inplace=True)
flux.index.names = ['pressure']
flux.columns = ['CRD', 'WGT']

pltdata = [{'srs':xr.DataArray(flux[m]),
            'label':f'{m}',
            'color':np.random.choice(all_palettes['Category10'][10]),
            'alpha':np.random.uniform(.5, .8),
            'line_width':np.random.uniform(3, 6),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for m in flux]

fig_lin = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='flux', prange=(50, 1050))
fig_log = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='flux', prange=(1e-2, 200), y_axis_type='log')
show(gridplot(fig_lin, fig_log, ncols=2))

# fin

In [59]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')